# Srock Strategy - Index Enhancement 0003

## 1. Import Library

In [1]:
import dai
import random
import pandas as pd

## 2. Prepare Strategy Data

In [2]:
sd = '2023-12-20'
ed = '2026-02-13'

In [3]:
sql_factor =  f"""
SELECT 
    date, 
    instrument,
    (c_rank(total_market_cap) * 0.5 + c_rank(-1*turn) * 0.3 + c_rank(close) * 0.2) AS factor,
FROM cn_stock_prefactors
WHERE is_risk_warning = 0
AND list_sector = 1
AND pe_ttm > 0
AND close < 50
AND is_hs300 = 1 
ORDER BY date, factor
"""

In [4]:
sql_trade = f"""
WITH
data_strategy AS (
    {sql_factor}
),
data_filter AS (
    SELECT
        date,
        instrument,
        factor AS score,
        c_rank(factor) AS score_rank,
    FROM data_strategy
    QUALIFY score_rank <= 10
),
data_date AS (
    SELECT
        date,
        instrument,
        score, 
        score_rank, 
        1 / c_sum(1) AS position, 
    FROM data_filter JOIN mldt_cn_stock_calendar_daily USING (date)
    WHERE is_month_end_trade = 1
)
SELECT *
FROM data_date
ORDER BY date, score_rank
"""

In [5]:
df_strategy = dai.query(sql_trade, filters = {"date":[sd, ed]}).df()
df_strategy

,date,instrument,score,score_rank,position
0,2023-12-29,601615.SH,10.6,1.0,0.1
1,2023-12-29,601878.SH,13.4,2.0,0.1
2,2023-12-29,601117.SH,17.0,3.0,0.1
3,2023-12-29,601901.SH,20.0,4.0,0.1
4,2023-12-29,601059.SH,20.3,5.0,0.1
...,...,...,...,...,...
255,2026-01-30,601059.SH,16.8,6.0,0.1
256,2026-01-30,601878.SH,17.2,7.0,0.1
257,2026-01-30,001391.SZ,17.8,8.0,0.1
258,2026-01-30,601456.SH,19.4,9.0,0.1


## 3. Backtest Strategy

In [6]:
from bigmodule import M

def BigTrader_Initialize(context):
    from bigtrader.finance.commission import PerOrder
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))

def BigTrader_Before_Trading(context, data):
    pass

def BigTrader_Handle_Tick(context, tick):
    pass

def BigTrader_Handle_Data(context, data):

    df_now = context.data[context.data["date"] == data.current_dt.strftime("%Y-%m-%d")]

    if len(df_now) == 0:
        return
    
    target_instruments  = list(df_now["instrument"])
    holding_instruments = list(context.get_account_positions().keys())

    for instrument in holding_instruments:
        if instrument not in target_instruments:
            context.order_target_percent(instrument, 0)
        
    for i, x in df_now.iterrows():
        position = 0.0 if pd.isnull(x.position) else float(x.position)
        context.order_target_percent(x.instrument, position)

def BigTrader_Handle_Trade(context, trade):
    pass

def BigTrader_Handle_Order(context, order):
    pass

def BigTrader_After_Trading(context, data):
    pass

BigTrader = M.bigtrader.v34(
    
    data = df_strategy,
    
    start_date = """""",
    end_date   = """""",
    
    initialize           = BigTrader_Initialize,
    before_trading_start = BigTrader_Before_Trading,
    handle_tick          = BigTrader_Handle_Tick,
    handle_data          = BigTrader_Handle_Data,
    handle_trade         = BigTrader_Handle_Trade,
    handle_order         = BigTrader_Handle_Order,
    after_trading        = BigTrader_After_Trading,
    
    capital_base = 500000  + random.uniform(0, 10),
    frequency="""daily""",
    product_type="""自动""",
    rebalance_period_type="""交易日""",
    rebalance_period_days="""1""",
    rebalance_period_roll_forward=True,
    backtest_engine_mode="""标准模式""",
    before_start_days=0,
    volume_limit=1,
    order_price_field_buy="""open""",
    order_price_field_sell="""open""",
    benchmark="""沪深300指数""",
    
    plot_charts=True,
    debug=False,
    backtest_only=False,
    m_name="""BigTrader"""
) 

[2026-02-18 20:02:15] [info     ] bigtrader.v34 开始运行 ..
[2026-02-18 20:02:15] [info     ] 2023-12-29, 2026-01-30, , , instruments=28
[2026-02-18 20:02:15] [info     ] bigtrader module V2.2.0
[2026-02-18 20:02:15] [info     ] bigtrader engine v0.1.0.post9+g7a244b6 2026-02-10
[2026-02-18 20:02:16] [info     ] backtest done, raw_perf_ds:dai.DataSource("_e3041bc5c47c482c97b02cba9894786e")


[2026-02-18 20:02:17] [info     ] bigtrader.v34 运行完成 [2.388s].
